# IT Support Chatbot - Interactive Notebook

This notebook demonstrates the RAG chatbot system with:
- Vector store querying
- LangChain integration
- Function calling
- End-to-end chatbot testing

## 1. Setup and Imports

In [ ]:
import os
import json
from dotenv import load_dotenv
from chatbot import ITSupportChatbot
from function_calling import (
    create_support_ticket, 
    check_ticket_status,
    check_system_status,
    search_employee_directory
)

# Load environment variables
load_dotenv()

print("✓ Imports successful")

## 2. Explore Knowledge Base

In [ ]:
# Load and display knowledge base
with open('it_knowledge_base.json', 'r') as f:
    kb = json.load(f)

print(f"Knowledge Base Statistics:")
print(f"Total Articles: {len(kb)}")
print(f"\nCategories:")

categories = {}
for article in kb:
    cat = article['category']
    categories[cat] = categories.get(cat, 0) + 1

for cat, count in categories.items():
    print(f"  • {cat}: {count} article(s)")

print("\nSample Article:")
print(f"ID: {kb[0]['id']}")
print(f"Title: {kb[0]['title']}")
print(f"Category: {kb[0]['category']}")
print(f"Tags: {', '.join(kb[0]['tags'])}")

## 3. Test Vector Store Retrieval

In [ ]:
from build_vector_store import VectorStoreBuilder

# Load vector store
builder = VectorStoreBuilder()
vector_store = builder.load_vector_store()

# Test queries
test_queries = [
    "password reset",
    "vpn not working",
    "printer offline",
    "email sync issue"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    results = vector_store.similarity_search(query, k=2)
    
    for i, doc in enumerate(results, 1):
        print(f"\nResult {i}:")
        print(f"Title: {doc.metadata['title']}")
        print(f"ID: {doc.metadata['id']}")
        print(f"Category: {doc.metadata['category']}")
        print(f"Preview: {doc.page_content[:200]}...")

## 4. Initialize Chatbot

In [ ]:
# Initialize the chatbot
chatbot = ITSupportChatbot()
print("Chatbot initialized and ready!")

## 5. Test Chatbot Responses

In [ ]:
def chat(message):
    """Helper function to chat with the bot"""
    print(f"\n{'='*70}")
    print(f"You: {message}")
    print(f"{'='*70}")
    
    response, sources, func_result = chatbot.process_message(message)
    
    print(f"\nIT Support: {response}")
    
    if sources:
        print(f"\n📚 Sources Used:")
        for source in sources[:2]:
            print(f"  • {source['title']} (ID: {source['id']})")
    
    return response, sources

# Test query 1
chat("How do I reset my password?")

In [ ]:
# Test query 2
chat("My computer is running very slow, what should I do?")

In [ ]:
# Test query 3
chat("I can't connect to the VPN")

In [ ]:
# Test query 4 - Follow-up question
chat("What if I still can't connect after trying those steps?")

## 6. Test Function Calling

In [ ]:
# Test creating a support ticket
print("Creating a support ticket...\n")

ticket = create_support_ticket(
    title="Cannot access shared drive",
    description="When I try to access \\\\fileserver\\projects, I get 'Access Denied' error. I need access to work on Q4 reports.",
    category="access",
    priority="high"
)

print("Ticket Created:")
print(json.dumps(ticket, indent=2))

In [ ]:
# Test checking ticket status
ticket_id = ticket['ticket_id']
print(f"Checking status of ticket: {ticket_id}\n")

status = check_ticket_status(ticket_id)
print(json.dumps(status, indent=2))

In [ ]:
# Test system status check
systems = ["email", "vpn", "file_server"]

for system in systems:
    status = check_system_status(system)
    print(f"\n{system.upper()}:")
    print(f"  Status: {status['status']}")
    print(f"  Uptime: {status.get('uptime', 'N/A')}")

In [ ]:
# Test employee directory search
print("Searching for IT Support staff...\n")

results = search_employee_directory(department="IT Support")

for emp in results:
    print(f"Name: {emp['name']}")
    print(f"Email: {emp['email']}")
    print(f"Phone: {emp['phone']}")
    print(f"Location: {emp['location']}")
    print("-" * 50)

## 7. Test Conversation Memory

In [ ]:
# Test multi-turn conversation
chat("I'm having trouble with my email")

In [ ]:
# Follow-up without context
chat("What's the mailbox size limit?")

In [ ]:
# Clear memory and test
chatbot.reset_conversation()
chat("What were we just talking about?")

## 8. Similarity Search Analysis

In [ ]:
# Analyze similarity scores
query = "my account is locked"

# Get results with scores
results_with_scores = vector_store.similarity_search_with_score(query, k=5)

print(f"Query: '{query}'\n")
print("Top 5 Results with Similarity Scores:\n")

for doc, score in results_with_scores:
    print(f"Score: {score:.4f}")
    print(f"Title: {doc.metadata['title']}")
    print(f"Category: {doc.metadata['category']}")
    print(f"ID: {doc.metadata['id']}")
    print("-" * 60)

## 9. Batch Testing

In [ ]:
# Batch test multiple queries
test_cases = [
    "How do I set up MFA?",
    "My Teams camera isn't working",
    "How to backup files?",
    "Reporting phishing emails",
    "Mobile email setup for iPhone"
]

results = []

for query in test_cases:
    response, sources, _ = chatbot.process_message(query)
    results.append({
        'query': query,
        'response_length': len(response),
        'sources_found': len(sources),
        'primary_source': sources[0]['title'] if sources else 'None'
    })
    
    # Reset for next query
    chatbot.reset_conversation()

# Display results
import pandas as pd
df = pd.DataFrame(results)
print("Batch Test Results:")
print(df.to_string(index=False))

## 10. Performance Metrics

In [ ]:
import time

# Measure response time
queries = [
    "password reset",
    "vpn setup",
    "printer not working"
]

times = []

for query in queries:
    start = time.time()
    response, _, _ = chatbot.process_message(query)
    elapsed = time.time() - start
    times.append(elapsed)
    print(f"Query: '{query}' - Response time: {elapsed:.2f}s")
    chatbot.reset_conversation()

print(f"\nAverage response time: {sum(times)/len(times):.2f}s")
print(f"Min: {min(times):.2f}s, Max: {max(times):.2f}s")

## Summary

This notebook demonstrated:
1. ✅ Vector store retrieval with FAISS
2. ✅ LangChain conversational chain
3. ✅ Function calling capabilities
4. ✅ Conversation memory
5. ✅ Performance metrics

The chatbot successfully:
- Retrieves relevant knowledge base articles
- Maintains conversation context
- Executes functions (tickets, status checks)
- Provides accurate IT support responses